In [12]:
import torch 
import torch.nn as nn

import sys
sys.path.append("../auto_LiRPA/")
from auto_LiRPA import BoundedModule, BoundedTensor, PerturbationLpNorm

sys.path.append("../complete_verifier/")
from arguments import ConfigHandler
from abcrown import ABCROWN


import onnx
from onnx2pytorch import ConvertModel

# Load ONNX Model

In [2]:
onnx_model = onnx.load("./models/net6x50_best.onnx")
model = ConvertModel(onnx_model, experimental=True)

/home/philipp/anaconda3/envs/abcrown-fork/lib/python3.11/site-packages/onnx2pytorch/convert/layer.py:29: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /opt/conda/conda-bld/pytorch_1708025845868/work/torch/csrc/utils/tensor_numpy.cpp:206.)
  layer.weight.data = torch.from_numpy(numpy_helper.to_array(weight))
/home/philipp/anaconda3/envs/abcrown-fork/lib/python3.11/site-packages/onnx2pytorch/convert/model.py:151: UserWarning: Using experimental implementation that allows 'batch_size > 1'.Batchnorm layers could potentially produce false outputs.
  warnings.warn(


In [3]:
model

ConvertModel(
  (Flatten_/0/Flatten_output_0): Flatten()
  (Gemm_/1/Gemm_output_0): Linear(in_features=12, out_features=50, bias=True)
  (Relu_/2/Relu_output_0): ReLU(inplace=True)
  (Gemm_/3/Gemm_output_0): Linear(in_features=50, out_features=50, bias=True)
  (Relu_/4/Relu_output_0): ReLU(inplace=True)
  (Gemm_/5/Gemm_output_0): Linear(in_features=50, out_features=50, bias=True)
  (Relu_/6/Relu_output_0): ReLU(inplace=True)
  (Gemm_/7/Gemm_output_0): Linear(in_features=50, out_features=50, bias=True)
  (Relu_/8/Relu_output_0): ReLU(inplace=True)
  (Gemm_/9/Gemm_output_0): Linear(in_features=50, out_features=50, bias=True)
  (Relu_/10/Relu_output_0): ReLU(inplace=True)
  (Gemm_/11/Gemm_output_0): Linear(in_features=50, out_features=50, bias=True)
  (Relu_/12/Relu_output_0): ReLU(inplace=True)
  (Gemm_Y): Linear(in_features=50, out_features=1, bias=True)
)

# Dataset Format

The dataset consists of $5000$ samples of $(l, u, \alpha)$ for $2\times 2$ maxpool units.

In [4]:
ds_val = torch.load("./datasets/maxpool2x2_val_100k_sorted.pth")
X, y = ds_val.tensors

print("X.shape = ", X.shape)
print("y.shape = ", y.shape)

X.shape =  torch.Size([5000, 3, 2, 2])
y.shape =  torch.Size([5000, 1])


The cell below shows, which entries of the model's input represent which quantities (the batch dimension doesn't have to be $5$, but can be an arbitrary value).

In [14]:
lua = X[:5,0:3,:,:]
l = X[:5,0:1,:,:]
u = X[:5,1:2,:,:]
alpha = X[:5,2:3,:,:]

x_in = torch.ones_like(l)

b = model(lua)
x_hat = l + (u - l)*x_in

# Reformulate Specification

In the specification, we ultimately want
$$
a^T \hat{x} + b \geq \hat{x}_i ~~ \forall i = 1,\dots,n
$$
We reorder the dimension to efficiently represent the dot products as a single matrix multiplication.

Denote by 
$$
r_{nc} = \sum_{h,w} a_{nchw} \hat{x}_{nchw}
$$
the dot product for each of the input neurons.

Since we sum over all values of $h, w$ anyways, we can also flatten the tensors beforehand to get the equivalent result
$$
r_{nc} = \sum_{z} a_{ncz} \hat{x}_{ncz}
$$
where $z \leq h \cdot w$.

If we reorder the dimensions of $\hat{x}$ to $\hat{x}^T_{nzc}$, then we have the formula for batched matrix multiplication
$$
r_{nc} = \sum_{z} a_{ncz} \hat{x}^T_{nzc}
$$


In [15]:
alpha_flat = alpha.flatten(-2)
x_hat_flat = x_hat.flatten(-2)
x_hat_T = x_hat_flat.transpose(-1, -2)

a_T_x = torch.matmul(alpha_flat, x_hat_T).squeeze(-1)
y = a_T_x + b

violation = x_hat.flatten(-3) - y 


# Define Model for Verification

$\alpha$-$\beta$-CROWN's parsing of ONNX files and PyTorch models seems to be dependent on the names of the parameters in the layers.
For that reason, we can't use submodules or `moduleList`.

Instead, we use a regular python list to store all layers of the `bias_model`.

For tracing, we need to ensure that the parameters of these layers, however, don't require a gradient.
So we have to set `requires_grad=False` for all of them.

The above approach doesn't seem to work due to 
- `onnx2pytorch` being unable to handle linear layers with no parameters (since they don't require a gradient) and
- $\alpha$-$\beta$-CROWN has some problems with the `BoundSlice` method.

In [5]:
class MaxPoolVerification_6Layers(nn.Module):

    def __init__(self, bias_model_layers):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = bias_model_layers[1]
        self.fc2 = bias_model_layers[3]
        self.fc3 = bias_model_layers[5]
        self.fc4 = bias_model_layers[7]
        self.fc5 = bias_model_layers[9]
        self.fc6 = bias_model_layers[11]
        self.fc7 = bias_model_layers[13]

    def forward(self, x):
        # l, u, alpha for all batches and for all of the w x h inputs
        lua = x[:,0:3,:,:]
        l = x[:,0:1,:,:]
        u = x[:,1:2,:,:]
        alpha = x[:,2:3,:,:]
        # normalized inputs to [0, 1] for all batches ans all w x h inputs
        x_in = x[:,3:4,:,:]

        # get bias prediction from model
        #b = self.bias_model(lua)
        b = b = self.flatten(lua)
        b = self.fc1(b)
        b = nn.functional.relu(b)
        b = self.fc2(b)
        b = nn.functional.relu(b)
        b = self.fc3(b)
        b = nn.functional.relu(b)
        b = self.fc4(b)
        b = nn.functional.relu(b)
        b = self.fc5(b)
        b = nn.functional.relu(b)
        b = self.fc6(b)
        b = nn.functional.relu(b)
        b = self.fc7(b)
        
        # transfrom input from [0, 1] to [l, u]
        x_hat = l + (u - l)*x_in 

        # we need alpha^T x_hat for all the batches 
        # so we manipulate the shapes to express it as matmul
        alpha_flat = alpha.flatten(-2)
        x_hat_flat = x_hat.flatten(-2)
        x_hat_T = x_hat_flat.transpose(-1, -2)
        a_T_x = torch.matmul(alpha_flat, x_hat_T)[:,:,0]
        y = a_T_x + b

        # we want alpha^T x_hat + b >= x_hat_i for all i
        # <==> 0 >= x_hat_i - (alpha^T x_hat  + b)
        # so we want the RHS to be smaller equal 0, if it is 
        # larger than 0, we have a violation!!!
        violation = x_hat.flatten(-3) - y 

        return violation

In [6]:
onnx_model = onnx.load("./models/net6x50_best.onnx")
model = ConvertModel(onnx_model, experimental=True)

mpv6 = MaxPoolVerification_6Layers(list(model.children()))

In [8]:
x = torch.zeros(1,4,2,2)
mpv6(x)

tensor([[-0.2598, -0.2598, -0.2598, -0.2598]], grad_fn=<SubBackward0>)

In [27]:
torch.onnx.export(mpv6, x, './models/mpv6.onnx', export_params=True, do_constant_folding=True, input_names=['X'], output_names=['Y'])

In [9]:
model_abcrown = BoundedModule(mpv6, x)

data_min = torch.zeros_like(x)
data_max = torch.ones_like(x)
center = 0.5 * (data_min + data_max)

ptb = PerturbationLpNorm(x_L=data_min, x_U=data_max)
bound_x = BoundedTensor(center, ptb)

In [10]:
lb, ub = model_abcrown.compute_bounds(x=(bound_x,), method='crown')
print("lb = ", lb)
print("ub = ", ub)

lb =  tensor([[-278.2891, -278.2891, -278.2891, -278.2891]], grad_fn=<ViewBackward0>)
ub =  tensor([[244.6781, 244.6781, 244.6781, 244.6781]], grad_fn=<ViewBackward0>)


In [11]:
lb, ub = model_abcrown.compute_bounds(x=(bound_x,), method='alpha-crown')
print("lb = ", lb)
print("ub = ", ub)

lb =  tensor([[-91.7884, -92.4998, -91.8478, -92.2516]])
ub =  tensor([[88.5101, 88.5159, 88.4735, 88.4429]])


# Verification with $\alpha$-$\beta$-CROWN

Use https://github.com/Verified-Intelligence/Lyapunov_Stable_NN_Controllers/blob/main/neural_lyapunov_training/levelset.py as example.

In [45]:
def write_vnnlib(filename, lbs, ubs, n_out):
    n_in = lbs.shape[0]
    with open(filename, "w") as f:
        f.write("; Input variables l, u, alpha, x\n")
        f.write(f"; l:     {0} -- {int(n_in / 4)}\n")
        f.write(f"; u:     {0} -- {int(n_in / 4)}\n")
        f.write(f"; alpha: {0} -- {int(n_in / 4)}\n")
        f.write(f"; x:     {0} -- {int(n_in / 4)}\n")
        for i, (lb, ub) in enumerate(zip(lbs, ubs)):
            f.write(f"(declare-const X_{i} Real)\n")

        f.write("\n")
        f.write("; Output variables (violations)\n")
        for i in range(n_out):
            f.write(f"(declare-const Y_{i} Real)\n")

        f.write("\n")
        f.write("; Input constraints\n")
        for i, (lb, ub) in enumerate(zip(lbs, ubs)):
            f.write(f"(assert (>= X_{i} {lb}))\n")
            f.write(f"(assert (<= X_{i} {ub}))\n")

        f.write("\n")

        f.write("; Counterexample found, if one of the violations is > 0\n")
        f.write("(assert (or\n")
        for i in range(n_out):
            f.write(f"\t(and (>= Y_{i} 0))\n")

        f.write("))")

In [46]:
def generate_csv(csv_path, onnx_path, vnnlib_path, timeout=100):
    with open(csv_path, "w") as f:
        f.write(f"{onnx_path},{vnnlib_path},{timeout}\n")

In [50]:
write_vnnlib("./verification/specs/mpv2x2.vnnlib", torch.zeros(4*4), torch.ones(4*4), 4)
generate_csv("./verification/specs/mpv2x2.csv", "./models/mpv6.onnx", "./specs/mpv2x2.vnnlib")

In [34]:
config = ConfigHandler()
config.parse_config(["--config=./configs/mpv.yaml"])

Configurations:

general:
  device: cpu
  seed: 100
  conv_mode: patches
  deterministic: false
  double_fp: true
  loss_reduction_func: sum
  sparse_alpha: true
  sparse_interm: true
  save_adv_example: false
  eval_adv_example: false
  show_adv_example: false
  precompile_jit: false
  complete_verifier: bab
  enable_incomplete_verification: true
  csv_name: specs/mpv2x2.csv
  results_file: mpv2x2_6x50_results.txt
  root_path: ..
  deterministic_opt: false
  graph_optimizer: 'Customized("custom_graph_optimizer", "default_optimizer")'
  buffer_has_batchdim: false
  save_output: true
  output_file: mpv2x2_6x50_out.pkl
  return_optimized_model: false
model:
  name: null
  path: null
  onnx_path: null
  onnx_path_prefix: ''
  cache_onnx_conversion: false
  debug_onnx: false
  onnx_quirks: null
  input_shape: [-1, 4, 2, 2]
  onnx_loader: default_onnx_and_vnnlib_loader
  onnx_optimization_flags: none
  onnx_vnnlib_joint_optimization_flags: none
  check_optmized: false
  flatten_final_output

Namespace(config='./configs/mpv.yaml', device='cuda', seed=100, conv_mode='patches', deterministic=False, double_fp=False, loss_reduction_func='sum', no_sparse_alpha=True, no_sparse_interm=True, save_adv_example=False, eval_adv_example=False, show_adv_example=False, precompile_jit=False, complete_verifier='bab', incomplete=True, csv_name=None, results_file='out.txt', root_path='', deterministic_opt=False, graph_optimizer='Customized("custom_graph_optimizer", "default_optimizer")', buffer_has_batchdim=False, save_output=False, output_file='out.pkl', return_optimized_model=False, model=None, load_model=None, onnx_path=None, onnx_path_prefix='', cache_onnx_conversion=False, debug_onnx=False, onnx_quirks=None, input_shape=None, onnx_loader='default_onnx_and_vnnlib_loader', onnx_optimization_flags='none', onnx_vnnlib_joint_optimization_flags='none', check_optmized=False, flatten_final_output=False, optimize_graph=None, model_with_jacobian=False, start=0, end=10000, select_instance=None, num

In [51]:
verifier = ABCROWN(["--config=./verification/mpv.yaml"])

Configurations:

general:
  device: cpu
  seed: 100
  conv_mode: patches
  deterministic: false
  double_fp: true
  loss_reduction_func: sum
  sparse_alpha: true
  sparse_interm: true
  save_adv_example: false
  eval_adv_example: false
  show_adv_example: false
  precompile_jit: false
  complete_verifier: bab
  enable_incomplete_verification: true
  csv_name: specs/mpv2x2.csv
  results_file: mpv2x2_6x50_results.txt
  root_path: ../jupyter/verification
  deterministic_opt: false
  graph_optimizer: 'Customized("custom_graph_optimizer", "default_optimizer")'
  buffer_has_batchdim: false
  save_output: true
  output_file: mpv2x2_6x50_out.pkl
  return_optimized_model: false
model:
  name: null
  path: null
  onnx_path: null
  onnx_path_prefix: ''
  cache_onnx_conversion: false
  debug_onnx: false
  onnx_quirks: null
  input_shape: [-1, 4, 2, 2]
  onnx_loader: default_onnx_and_vnnlib_loader
  onnx_optimization_flags: none
  onnx_vnnlib_joint_optimization_flags: none
  check_optmized: false
 

In [52]:
verifier.main()

Experiments at Mon Feb 10 16:34:48 2025 on polyphem
no customized start/end sample, testing all samples in specs/mpv2x2.csv
Internal results will be saved to mpv2x2_6x50_results.txt.

 %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% idx: 0, vnnlib ID: 0 %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
Using onnx ./models/mpv6.onnx
Using vnnlib ./specs/mpv2x2.vnnlib
16 inputs and 4 outputs in vnnlib
Loading onnx ../jupyter/verification/./models/mpv6.onnx wih quirks {}

*************Error traceback*************
Traceback (most recent call last):
  File "/home/philipp/VerifyNN/alpha-beta-Crown-fork/jupyter/../complete_verifier/load_model.py", line 191, in load_model_onnx
    output_onnx = inference_onnx(path, x.numpy())
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/philipp/VerifyNN/alpha-beta-Crown-fork/jupyter/../complete_verifier/load_model.py", line 114, in inference_onnx
    res = sess.run(None, {sess.get_inputs()[0].name: input})[0]
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

/home/philipp/anaconda3/envs/abcrown-fork/lib/python3.11/site-packages/onnx2pytorch/convert/model.py:151: UserWarning: Using experimental implementation that allows 'batch_size > 1'.Batchnorm layers could potentially produce false outputs.
  warnings.warn(
/home/philipp/VerifyNN/alpha-beta-Crown-fork/jupyter/../complete_verifier/load_model.py:200: UserWarning: Not able to check model's conversion correctness
  warnings.warn('Not able to check model\'s conversion correctness')
/home/philipp/anaconda3/envs/abcrown-fork/lib/python3.11/site-packages/onnx2pytorch/operations/slice.py:73: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert (steps == 1 or steps == -1) and axes == int(axes) and start == int(start) and end == int(end)
/home/philipp/anaconda3/envs/abcrown-fork/l

Model: BoundedModule(
  (/0): BoundInput(name=/0, inputs=[], perturbed=True)
  (/15): BoundBuffers(name=/15, inputs=[], perturbed=False)
  (/36): BoundParams(name=/36, inputs=[], perturbed=False)
  (/37): BoundParams(name=/37, inputs=[], perturbed=False)
  (/38): BoundParams(name=/38, inputs=[], perturbed=False)
  (/39): BoundParams(name=/39, inputs=[], perturbed=False)
  (/40): BoundParams(name=/40, inputs=[], perturbed=False)
  (/41): BoundParams(name=/41, inputs=[], perturbed=False)
  (/42): BoundParams(name=/42, inputs=[], perturbed=False)
  (/43): BoundParams(name=/43, inputs=[], perturbed=False)
  (/44): BoundParams(name=/44, inputs=[], perturbed=False)
  (/45): BoundParams(name=/45, inputs=[], perturbed=False)
  (/46): BoundParams(name=/46, inputs=[], perturbed=False)
  (/47): BoundParams(name=/47, inputs=[], perturbed=False)
  (/48): BoundParams(name=/48, inputs=[], perturbed=False)
  (/49): BoundParams(name=/49, inputs=[], perturbed=False)
  (/53): BoundBuffers(name=/53, input

AssertionError: 